# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
import sys, os

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import IntegerType, ArrayType, FloatType, DoubleType, Row, DateType
from pyspark.ml.linalg import DenseVector, SparseVector
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import  MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
import re
import copy
import time
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest

sns.set(style="whitegrid")

In [3]:
spark = SparkSession.builder.appName('sparkify') \
                    .config("spark.driver.memory", "32g") \
                    .config("spark.driver.maxResultSize", "64g") \
                    .config("spark.executor.memory", "16g") \
                    .config("spark.executor.cores", "5") \
                    .config("spark.executor.instances", "60") \
                    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
eventDf = spark.read.json("mini_sparkify_event_data.json")

In [5]:
eventDf.where(col("auth")==lit("Guest")).select("userId").distinct().show()

+------+
|userId|
+------+
|      |
+------+



In [6]:
def clean(df):
    """cleans dataframe
    :param df: Spark DataFrame
    :return df cleaned Spark DataFrame
    """
    df = df.where((col("userId")!=lit(''))\
                              &(col("userId").isNotNull())\
                              &(lower(col("userId"))!=lit('null'))\
                              &(col("sessionId").cast('string')!=lit(''))\
                              &(col("sessionId").isNotNull())\
                              &(lower(col("sessionId"))!=lit('null')))

    for field in df.schema.fields:
        if field.dataType==StringType():
            df = df.withColumn(field.name, regexp_replace(field.name, '[^a-zA-Z0-9\,\-]', ''))

    df = df.withColumn('interaction_time', from_unixtime(col('ts').cast(LongType())/1000).cast(TimestampType()))
    df = df.withColumn('month', month(col('interaction_time')))
    df = df.withColumn('date', from_unixtime(col('ts')/1000).cast(DateType()))
    df = df.filter(col('auth')!='LoggedOut')
    df = df.withColumn('location', split(col('location'),',').getItem(1))
    return df

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [7]:
eventDf.groupBy("level").count().orderBy("level").show()

+-----+------+
|level| count|
+-----+------+
| free| 58338|
| paid|228162|
+-----+------+



In [8]:
eventDf.groupBy("auth").count().orderBy("auth").show()

+----------+------+
|      auth| count|
+----------+------+
| Cancelled|    52|
|     Guest|    97|
| Logged In|278102|
|Logged Out|  8249|
+----------+------+



In [9]:
eventDf.groupBy("page").count().orderBy("page").show(30)

+--------------------+------+
|                page| count|
+--------------------+------+
|               About|   924|
|          Add Friend|  4277|
|     Add to Playlist|  6526|
|              Cancel|    52|
|Cancellation Conf...|    52|
|           Downgrade|  2055|
|               Error|   258|
|                Help|  1726|
|                Home| 14457|
|               Login|  3241|
|              Logout|  3226|
|            NextSong|228108|
|            Register|    18|
|         Roll Advert|  3933|
|       Save Settings|   310|
|            Settings|  1514|
|    Submit Downgrade|    63|
| Submit Registration|     5|
|      Submit Upgrade|   159|
|         Thumbs Down|  2546|
|           Thumbs Up| 12551|
|             Upgrade|   499|
+--------------------+------+



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [10]:
def label_n_transform(df):
    """transforms features from existing fields & adds churn column
    :param df: imput data
    :return df: transformed data
    """
    churnDf = df.withColumn("cancel", max(when((col('page').isin(['Cancellation Confirmation','Cancel']))|(col("auth")==lit('Cancelled')),lit(1)).otherwise(lit(0))).over(Window.partitionBy("userId")))\
                .withColumn("next_level", lead("level", 1).over(Window.partitionBy("userId").orderBy(col("ts").asc())))\
                .withColumn("downgrade", max(when((col("level")==lit('paid'))&(col("next_level")==lit('free')),lit(1)).otherwise(lit(0))).over(Window.partitionBy("userId")))\
                .withColumn("churn", when((col("downgrade")==lit(1))|(col("cancel")==lit(1)),lit(1)).otherwise(lit(0)))\
                .withColumn("latestInteraction", max(col('ts')).over(Window.partitionBy("userId")))\
                .withColumn("lastInteraction", max(when(col('churn')==lit(1),col('ts')).otherwise(lit(None))).over(Window.partitionBy("userId")))\
                .withColumn('registeredDays', ((coalesce(col('lastInteraction'),col('latestInteraction'))-col('registration'))/86400000).cast(IntegerType()))\
                .withColumn('validLevel', first('level').over(Window.partitionBy("userId").orderBy(col("ts").desc())))\
                .withColumn('avgLength', avg('length').over(Window.partitionBy("userId")))\
                .withColumn('cntSessions', size(collect_set("sessionId").over(Window.partitionBy("userId"))))\
                .withColumn('avgSessions', size(collect_set("sessionId").over(Window.partitionBy("userId",'date'))))\
                .withColumn('cntActiveDays', size(collect_set("date").over(Window.partitionBy("userId"))))\
                .withColumn('avgDailySessions', sum(col('avgSessions')).over(Window.partitionBy("userId"))/col('cntSessions'))\
                .withColumn('avgDailyItem', (max("itemInSession").over(Window.partitionBy("userId"))/col('cntActiveDays')))\
                .withColumn('avgSessionLength', (max("ts").over(Window.partitionBy('userId','date','sessionId'))-min("ts").over(Window.partitionBy('userId','date','sessionId')))*0.001)\
                .withColumn('avgDailySessionLength', sum(col('avgSessionLength')).over(Window.partitionBy("userId"))/col('cntSessions'))

    userChurnDf=churnDf.select('userId','gender', 'location','registeredDays', 'validLevel', 'avgLength', 'cntSessions', 'avgDailySessions', 'avgDailyItem', 'cntActiveDays','avgDailySessionLength','churn' ).dropDuplicates()
    
    userPageDf=churnDf.groupby('userId','date','cntActiveDays').pivot("page").agg(count(lit(1))).drop('CancellationConfirmation', 'Cancel')
    for c in userPageDf.columns[3:]:
        userPageDf=userPageDf.withColumn(c, (sum(coalesce(col(c),lit(0))).over(Window.partitionBy("userId")))/col('cntActiveDays'))\
                             .withColumnRenamed(c, 'avgDaily'+c)
    userPageDf=userPageDf.drop('date', 'cntActiveDays')
    
    df = userChurnDf.join(userPageDf, on='userId')
    churned_user_count = churnDf.filter(churnDf["churn"] == 1).select("userId").dropDuplicates().count()
    print("churned user count: {} (total: {})".format(churned_user_count, churnDf.count()))
    print("churned user rate: {:.2f}%".format(churned_user_count / churnDf.count() * 100))
    
    return churnDf, df
    

In [11]:
churnDf, finalDf = label_n_transform(clean(eventDf))

churned user count: 92 (total: 278154)
churned user rate: 0.03%


In [12]:
fig, ax = plt.subplots(figsize=(10,5))
page = churnDf.select(["page","churn"]).groupby([ "page","churn"]).count().sort("page").toPandas()
sns.barplot( x="count",y="page", hue="churn", data=page)
plt.savefig('page3.jpg')
plt.close()

In [13]:
fig, ax = plt.subplots(figsize=(10,5))
gender = churnDf.select(["userId","gender","churn"]).dropDuplicates().groupby([ "gender","churn"]).count().sort("gender").toPandas()
sns.barplot( x="count",y="gender", hue="churn", data=gender)
plt.savefig('gender3.jpg')
plt.close()

![](page3.jpg)![](gender3.jpg)

In [14]:
pd.DataFrame(finalDf.take(5), columns=finalDf.columns).head(10)

,userId,gender,location,registeredDays,validLevel,avgLength,cntSessions,avgDailySessions,avgDailyItem,cntActiveDays,...,avgDailyLogout,avgDailyNextSong,avgDailyRollAdvert,avgDailySaveSettings,avgDailySettings,avgDailySubmitDowngrade,avgDailySubmitUpgrade,avgDailyThumbsDown,avgDailyThumbsUp,avgDailyUpgrade
0,100010,F,CT,55,free,243.421445,7,54.428571,15.857143,7,...,0.714286,39.285714,7.428571,0.0,0.0,0.0,0.0,0.714286,2.428571,0.285714
1,100010,F,CT,55,free,243.421445,7,54.428571,15.857143,7,...,0.714286,39.285714,7.428571,0.0,0.0,0.0,0.0,0.714286,2.428571,0.285714
2,100010,F,CT,55,free,243.421445,7,54.428571,15.857143,7,...,0.714286,39.285714,7.428571,0.0,0.0,0.0,0.0,0.714286,2.428571,0.285714
3,100010,F,CT,55,free,243.421445,7,54.428571,15.857143,7,...,0.714286,39.285714,7.428571,0.0,0.0,0.0,0.0,0.714286,2.428571,0.285714
4,100010,F,CT,55,free,243.421445,7,54.428571,15.857143,7,...,0.714286,39.285714,7.428571,0.0,0.0,0.0,0.0,0.714286,2.428571,0.285714


# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [15]:
def build_pipeline(num_cols):
    """creates a pipeline w/ indexer & assembler
    :param num_cols:  list of numerical column names
    :return processing pipeline
    """
    idxr_location = StringIndexer(inputCol='location', outputCol='location_idx')
    idxr_gender = StringIndexer(inputCol='gender', outputCol='gender_idx')
    idxr_validLevel = StringIndexer(inputCol='validLevel', outputCol='validLevel_idx')

    vAssembler = VectorAssembler(inputCols=num_cols, outputCol='features')

    process_pipeline = Pipeline(stages=[idxr_location, idxr_gender, idxr_validLevel, vAssembler])

    return process_pipeline

In [16]:
def finalize_model_data (df):
    """transforms features for modelling
    :param features_df: feature dataframe
    :return model data
    """
    num_cols = []
    
    for field in df.schema.fields :
        if field.dataType!=StringType():
            num_cols.append(field.name)

    num_cols.remove('churn')

    process_pipeline = build_pipeline(num_cols)
    model_df = process_pipeline.fit(df).transform(df)

    return model_df

In [17]:
def evaluate_model(results):
    """output evaluator
    :param results: model prediction outputs
    :return None
    """

    f1_score_evaluator = MulticlassClassificationEvaluator(metricName='f1')
    f1_score = f1_score_evaluator.evaluate(results.select(col('label'), col('prediction')))
    print('F1 score : {:.2%}'.format(f1_score)) 
    
    auc_evaluator = BinaryClassificationEvaluator()
    metric_value = auc_evaluator.evaluate(results, {auc_evaluator.metricName: "areaUnderROC"})
    print('areaUnderROC : {:.2%}'.format(metric_value))

In [18]:
def fit_model(train, test):
    """fits and runs predictions 
    :param train : training data
    :param test : testing data
    :return 
    """
    train=train.withColumnRenamed('churn','label')
    test=test.withColumnRenamed('churn','label')

    ml = GBTClassifier()
    clf = ml.fit(train)
    print ("fitted")
    results = clf.transform(test)
    print ("transformed")
    evaluate_model(results)
    return clf, results

In [19]:
model_data = finalize_model_data(finalDf)

In [20]:
train, test = model_data.randomSplit([0.8, 0.2], seed=42)

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.

In [21]:
fit_model(train, test)

fitted
transformed
F1 score : 99.38%
areaUnderROC : 100.00%


(GBTClassificationModel (uid=GBTClassifier_69e734cf9ba2) with 20 trees,
 DataFrame[userId: string, gender: string, location: string, registeredDays: int, validLevel: string, avgLength: double, cntSessions: int, avgDailySessions: double, avgDailyItem: double, cntActiveDays: int, avgDailySessionLength: double, label: int, avgDailyAbout: double, avgDailyAddFriend: double, avgDailyAddtoPlaylist: double, avgDailyDowngrade: double, avgDailyError: double, avgDailyHelp: double, avgDailyHome: double, avgDailyLogout: double, avgDailyNextSong: double, avgDailyRollAdvert: double, avgDailySaveSettings: double, avgDailySettings: double, avgDailySubmitDowngrade: double, avgDailySubmitUpgrade: double, avgDailyThumbsDown: double, avgDailyThumbsUp: double, avgDailyUpgrade: double, location_idx: double, gender_idx: double, validLevel_idx: double, features: vector, rawPrediction: vector, probability: vector, prediction: double])